## For Second project of Strawberry image labeling
> **Idea: get information about number of object in each images using YOLOv8 medium model(trained)** <br>
> 이전 500장 라벨링 데이터를 통해 훈련시킨 YOLOv8모델을 통해 이미지에 어떤 객체가 얼마나 있는지 대략적인 정보를 얻고 <br>
> 정보를 바탕으로 이미지 1500장 선별, 1500장 -> 1000장 수렴 계획


In [73]:
from ultralytics import YOLO
from PIL import Image
import cv2

#n= 1000 
File_path=[]
File_names=[]
label_names = ['Flower','Before_blooming','Receptacle','Green_small_fruit','Strawberry_1','Strawberry_2','Strawberry_3']
N_flower=[]
N_fold_flower=[]
N_receptacle=[]
N_greenSF=[]
N_str1=[]
N_str2=[]
N_str3=[]
N_target=[]

model = YOLO("/home/hyeonji/YOLOv8/agtech_lab/Y8_medium_Result/weights/Y8_str_medium_best.pt")
model_D2 = YOLO("/home/hyeonji/YOLOv8/runs/detect/ALARAD_y8_m_bs8_D2/weights/best.pt")

In [74]:
def add_list():
    N_flower.append(0)
    N_fold_flower.append(0)
    N_receptacle.append(0)
    N_greenSF.append(0)
    N_str1.append(0)
    N_str2.append(0)
    N_str3.append(0)
    N_target.append(0)
    
def box_class_counter(results, idx):
    box_class = -1       
    cnt = 0
    add_list()
    #print("----------------",idx)
    
    for r in results.boxes:
                
        box_class = int(r.cls.tolist()[0])
        
        if box_class == 0:
            N_flower[idx] +=1
            N_target[idx] += 1

        elif box_class == 1:
            N_fold_flower[idx] +=1
            N_target[idx] += 1

        elif box_class == 2:
            N_receptacle[idx] +=1
            N_target[idx] += 1

        elif box_class == 3:
            N_greenSF[idx] +=1
            N_target[idx] += 1

        elif box_class == 4:
            N_str1[idx] +=1
            N_target[idx] += 10

        elif box_class == 5:
            N_str2[idx] +=1
            N_target[idx] += 10

        elif box_class == 6:
            N_str3[idx] +=1
            N_target[idx] += 10
        else:
            print(box_class)
            
    

In [75]:
import pandas as pd
import os
path = "/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496/"

Date = []  # 촬영일
Time = []  # 촬영시
Line = []  # 촬영 라인
View = []  # Topview, sideview
Position = []  # 촬영위치
filename = []  # 파일명
Rootdirs = []  # 추후 파일 찾기 편하려고 만든 루트
D_N = []  # 오전 오후 구분


for root, subdirs, files in os.walk(path):
    for i in files:
            if len(i.split('_')) > 1:
                try:
                    Date.append(i.split('_')[0])
                    Time.append(i.split('_')[1])
                    if int(i.split('_')[1]) > 160000:
                        D_N.append(1)
                    else:
                        D_N.append(0)  # 낮
                    Line.append(i.split('_')[2])
                    View.append(i.split('_')[3])
                    Position.append(i.split('_')[4])
                    filename.append(i)
                    Rootdirs.append(root)

                except:
                    print(i)
#날짜 시간순 정렬 필 (그러면 원래 디렉토리내 파일 순서오 ㅏ같아짐)
myData = {
    'filename': filename, 'Date': Date, 'Time': Time, 'Line': Line, 'View': View, 'Position': Position, 'Rootdirs': Rootdirs, 'isDay': D_N
}
myDF = pd.DataFrame(myData)
myDF = myDF.sort_values(by=['Date','Time'],ignore_index=True)
myDF

,filename,Date,Time,Line,View,Position,Rootdirs,isDay
0,221217_082642_L1_V2_P11_D0410.png,221217,082642,L1,V2,P11,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,0
1,221217_082954_L1_V2_P18_D0690.png,221217,082954,L1,V2,P18,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,0
2,221217_162758_L1_V2_P17_D0650.png,221217,162758,L1,V2,P17,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
3,221217_163138_L1_V2_P25_D0970.png,221217,163138,L1,V2,P25,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
4,221217_163301_L1_V2_P28_D1090.png,221217,163301,L1,V2,P28,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
...,...,...,...,...,...,...,...,...
995,230312_162438_L1_V2_P10_D0370.png,230312,162438,L1,V2,P10,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
996,230312_162535_L1_V2_P12_D0450.png,230312,162535,L1,V2,P12,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
997,230312_164227_L4_V2_P10_D0370.png,230312,164227,L4,V2,P10,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1
998,230312_164934_L4_V2_P23_D0890.png,230312,164934,L4,V2,P23,/home/hyeonji/New_storage/alarad_img_P2_1000/V...,1


In [4]:
results = model.predict(source='/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496/', conf=0.2, show=False)


Ultralytics YOLOv8.0.8 🚀 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10008MiB)
Fusing layers... 
Model summary: 218 layers, 25843813 parameters, 0 gradients, 78.7 GFLOPs


In [61]:
results = model_D2.predict(source='/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496/', conf=0.33, show=False, save_txt=False)


Ultralytics YOLOv8.0.8 🚀 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10008MiB)
Fusing layers... 
Model summary: 218 layers, 25843813 parameters, 0 gradients, 78.7 GFLOPs


In [87]:
results = model_D2.predict(source='/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496/230130_083057_L3_V2_P20_D0860.png', conf=0.2, show=False,save=True, save_txt=False)


Ultralytics YOLOv8.0.8 🚀 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10008MiB)
Fusing layers... 
Model summary: 218 layers, 25843813 parameters, 0 gradients, 78.7 GFLOPs
Results saved to /home/hyeonji/YOLOv8/runs/detect/predict2


In [88]:
idx = 0
for result in results:
    #하나의 이미지 당,, result 
    box_class_counter(result,idx)
    idx += 1

In [77]:
import openpyxl

#결과 엑셀화
img_result = {'filename': myDF['filename'], 'Date': myDF['Date'], 'Time': myDF['Time'], 'Line': myDF['Line'], 'D_N': myDF['isDay'],'flowers': N_flower, 'before_blooming': N_fold_flower, 'receptacle':N_receptacle, 'green_small_fruit':N_greenSF, 'strawberry1':N_str1, 'strawberry2':N_str2, 'strawberry3':N_str3, 'N_target':N_target }

ir_DF=pd.DataFrame(img_result)
ir_DF.to_excel('/home/hyeonji/New_storage/alarad_img_P2_1000/1000_image_info_for_New_testset_D2_230727.xlsx')

In [78]:
ir_DF = ir_DF.sort_values(by='N_target', ascending=False)

In [79]:
import copy

myDF = copy.deepcopy(ir_DF)

In [80]:
t1_pool_list1 = myDF.loc[(myDF['Line'] == 'L1') ]
t1_pool_list2 = myDF.loc[(myDF['Line'] == 'L2') ]
t1_pool_list3 = myDF.loc[(myDF['Line'] == 'L3')]
t1_pool_list4 = myDF.loc[(myDF['Line'] == 'L4')]


In [91]:
t1_pool_list4.reset_index(inplace=True)


,index,filename,Date,Time,Line,...,green_small_fruit,strawberry1,strawberry2,strawberry3,N_target
0,958,230309_164802_L4_V2_P20_D0770.png,230309,164802,L4,...,4,12,9,0,221
1,986,230311_164545_L4_V2_P16_D0610.png,230311,164545,L4,...,10,17,2,0,204
2,999,230312_165007_L4_V2_P24_D0930.png,230312,165007,L4,...,4,10,5,3,191
3,998,230312_164934_L4_V2_P23_D0890.png,230312,164934,L4,...,7,6,9,1,170
4,919,230304_165555_L4_V2_P23_D0890.png,230304,165555,L4,...,3,11,2,1,169
...,...,...,...,...,...,...,...,...,...,...,...
246,296,230116_164828_L4_V2_P20_D0770.png,230116,164828,L4,...,1,0,0,0,11
247,138,230103_152606_L4_V2_P13_D0490.png,230103,152606,L4,...,0,0,0,1,10
248,160,230105_123142_L4_V2_P13_D0490.png,230105,123142,L4,...,0,0,0,1,10
249,222,230110_083418_L4_V2_P11_D0410.png,230110,083418,L4,...,1,0,0,0,10


In [95]:
t1_pool_list4.to_excel('/home/hyeonji/ALARAD_P2_1000_50_L4.xlsx')

In [101]:
L4_25=t1_pool_list4.iloc[:25,1].to_list()
L4_25

['230309_164802_L4_V2_P20_D0770.png',
 '230311_164545_L4_V2_P16_D0610.png',
 '230312_165007_L4_V2_P24_D0930.png',
 '230312_164934_L4_V2_P23_D0890.png',
 '230304_165555_L4_V2_P23_D0890.png',
 '230304_164526_L4_V2_P04_D0130.png',
 '230309_165225_L4_V2_P28_D1090.png',
 '230307_170056_L4_V2_P19_D0730.png',
 '230311_164829_L4_V2_P21_D0810.png',
 '230309_164835_L4_V2_P21_D0810.png',
 '230305_165154_L4_V2_P27_D1050.png',
 '230310_085040_L4_V2_P25_D0970.png',
 '230308_164940_L4_V2_P23_D0890.png',
 '230221_164727_L4_V2_P19_D0730.png',
 '230309_084902_L4_V2_P22_D0850.png',
 '230211_085118_L4_V2_P26_D1010.png',
 '230302_164831_L4_V2_P21_D0810.png',
 '230303_165045_L4_V2_P25_D0970.png',
 '230218_165117_L4_V2_P26_D1010.png',
 '230308_084332_L4_V2_P12_D0450.png',
 '230302_163910_L4_V2_P04_D0130.png',
 '230228_085218_L4_V2_P28_D1090.png',
 '230206_165118_L4_V2_P26_D1010.png',
 '230305_083909_L4_V2_P04_D0130.png',
 '230308_163942_L4_V2_P05_D0170.png']

In [102]:
picked_img = [] #L3_20
picked_img = L1_25 + L2_25 + L4_25
picked_img

['230310_082415_L1_V2_P09_D0330.png',
 '230311_162216_L1_V2_P05_D0170.png',
 '230308_084320_L1_V2_P26_D1010.png',
 '230228_175239_L1_V2_P24_D0930.png',
 '230310_083106_L1_V2_P23_D0890.png',
 '230312_082649_L1_V2_P14_D0530.png',
 '230305_165008_L1_V2_P05_D0170.png',
 '230310_082248_L1_V2_P06_D0210.png',
 '230310_082318_L1_V2_P07_D0250.png',
 '230310_082642_L1_V2_P14_D0530.png',
 '230307_163750_L1_V2_P10_D0370.png',
 '230310_082940_L1_V2_P20_D0770.png',
 '230312_082940_L1_V2_P20_D0770.png',
 '230310_162310_L1_V2_P07_D0250.png',
 '230312_162147_L1_V2_P04_D0130.png',
 '230310_163211_L1_V2_P26_D1010.png',
 '230217_162451_L1_V2_P10_D0370.png',
 '230302_083003_L1_V2_P19_D0730.png',
 '230310_162242_L1_V2_P06_D0210.png',
 '230302_163532_L1_V2_P20_D0770.png',
 '230216_082657_L1_V2_P14_D0530.png',
 '230310_163143_L1_V2_P25_D0970.png',
 '230311_082252_L1_V2_P06_D0210.png',
 '230128_080949_L1_V2_P11_D0410.png',
 '230213_162518_L1_V2_P11_D0410.png',
 '230305_172010_L2_V2_P17_D0740.png',
 '230302_084

In [104]:
import os, shutil
import pandas as pd

lists=[]

for i in range(len(picked_img)):
    lists.append('/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496'+'/'+picked_img[i])


for i in range(len(lists)):
    shutil.copy(lists[i],'/home/hyeonji/ALARAD_New_testset/'+ picked_img[i])